# 訓練模型, 省時間, 複製別人間單的範例即可

In [19]:
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, AdamW, WarmupLinearSchedule

import csv
import json
import numpy as np
import pandas as pd


# 超参数
EPOCHS = 1  # 训练的轮数
BATCH_SIZE = 4  # 批大小
MAX_LEN = 10  # 文本最大长度
LR = 1e-5  # 学习率
WARMUP_STEPS = 100  # 热身步骤
T_TOTAL = 1000  # 总步骤

# pytorch的dataset类 重写getitem,len方法
class Custom_dataset(Dataset):
    def __init__(self, dataset_list):
        self.dataset = dataset_list

    def __getitem__(self, item):
        text = self.dataset[item][0]
        label = self.dataset[item][1]

        return text, label

    def __len__(self):
        return len(self.dataset)


# 加载数据集
def load_dataset(filepath, max_len):
    dataset_list = []
    f = open(filepath, 'r', encoding='utf-8')
    r = csv.reader(f)
    for item in r:
        if r.line_num == 1:
            continue
        dataset_list.append(item)
    
    # 根据max_len参数进行padding
    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    for item in dataset_list:
        item[0] = item[0].replace(' ','')
        num = max_len - len(item[0])
        if num < 0:
            item[0] = item[0][:max_len]
            item[0] = tokenizer.encode(item[0]) 
            num_temp = max_len - len(item[0])
            if num_temp > 0:
                for _ in range(num_temp):
                    item[0].append(0)
            # 在开头和结尾加[CLS] [SEP]
            item[0] = [101] + item[0] + [102]
            item[0] = str(item[0])
            continue

        for _ in range(num):
            item[0] = item[0] + '[PAD]'
        item[0] = tokenizer.encode(item[0])
        num_temp = max_len - len(item[0])
        if num_temp > 0:
            for _ in range(num_temp):
                item[0].append(0)
        item[0] = [101] + item[0] + [102]
        item[0] = str(item[0])

    return dataset_list


# 计算每个batch的准确率
def  batch_accuracy(pre, label):
    pre = pre.argmax(dim=1)
    correct = torch.eq(pre, label).sum().float().item()
    accuracy = correct / float(len(label))

    return accuracy


if __name__ == "__main__":

    # 生成数据集以及迭代器
    train_dataset = load_dataset('/home/jacklee/ocr/food_classification/train_data/train_data.csv', max_len = MAX_LEN)  # 7337 * 3
    test_dataset = load_dataset('/home/jacklee/ocr/food_classification/train_data/test_data.csv', max_len = MAX_LEN)  # 7356 * 3
  
    train_cus = Custom_dataset(train_dataset)
    train_loader = DataLoader(dataset=train_cus, batch_size=BATCH_SIZE, shuffle=False)

    # Bert模型以及相关配置
    config = BertConfig.from_pretrained('bert-base-chinese')
    config.num_labels = 2
    model = BertForSequenceClassification(config = config)
    model = BertForSequenceClassification.from_pretrained('bert-base-chinese', config=config)
    model.cuda()

 
    optimizer = AdamW(model.parameters(), lr=LR, correct_bias=False)
    scheduler = WarmupLinearSchedule(optimizer, warmup_steps = WARMUP_STEPS, t_total = T_TOTAL)

    # optimizer = optim.Adam(model.parameters(), lr=LR)

    model.train()
    print('开始训练...')
    for epoch in range(EPOCHS):
        for text, label in train_loader:
            text_list = list(map(json.loads, text))
            label_list = list(map(json.loads, label))
            
            text_tensor = torch.tensor(text_list).cuda()
            label_tensor = torch.tensor(label_list).cuda()

            outputs = model(text_tensor, labels=label_tensor)
            loss, logits = outputs[:2]
            optimizer.zero_grad()
            loss.backward()
            scheduler.step()
            optimizer.step()

            acc = batch_accuracy(logits, label_tensor)
            print('epoch:{} | acc:{} | loss:{}'.format(epoch, acc, loss))

    torch.save(model.state_dict(), 'bert_cla.ckpt')
    print('保存训练完成的model...')




开始训练...
epoch:0 | acc:1.0 | loss:0.30297398567199707
epoch:0 | acc:1.0 | loss:0.36378350853919983
epoch:0 | acc:1.0 | loss:0.35447001457214355
epoch:0 | acc:0.75 | loss:0.6050809621810913
epoch:0 | acc:0.75 | loss:0.5708495378494263
epoch:0 | acc:1.0 | loss:0.2989423871040344
epoch:0 | acc:1.0 | loss:0.26403796672821045
epoch:0 | acc:1.0 | loss:0.24721144139766693
epoch:0 | acc:0.75 | loss:0.607689619064331
epoch:0 | acc:0.75 | loss:0.562098503112793
epoch:0 | acc:1.0 | loss:0.23412813246250153
epoch:0 | acc:1.0 | loss:0.21824197471141815
epoch:0 | acc:0.75 | loss:0.5723121762275696
epoch:0 | acc:1.0 | loss:0.13467970490455627
epoch:0 | acc:0.75 | loss:0.6469825506210327
epoch:0 | acc:1.0 | loss:0.11735144257545471
epoch:0 | acc:0.5 | loss:0.9911912679672241
epoch:0 | acc:1.0 | loss:0.10627524554729462
epoch:0 | acc:1.0 | loss:0.13061529397964478
epoch:0 | acc:1.0 | loss:0.10724890232086182
epoch:0 | acc:1.0 | loss:0.10682299733161926
epoch:0 | acc:1.0 | loss:0.09847307205200195
epoch:

epoch:0 | acc:1.0 | loss:0.034328848123550415
epoch:0 | acc:1.0 | loss:0.006967723369598389
epoch:0 | acc:1.0 | loss:0.016558855772018433
epoch:0 | acc:0.75 | loss:1.2550219297409058
epoch:0 | acc:1.0 | loss:0.011753469705581665
epoch:0 | acc:1.0 | loss:0.004686117172241211
epoch:0 | acc:1.0 | loss:0.006470322608947754
epoch:0 | acc:1.0 | loss:0.009973883628845215
epoch:0 | acc:1.0 | loss:0.01245260238647461
epoch:0 | acc:1.0 | loss:0.006226718425750732
epoch:0 | acc:1.0 | loss:0.0060836076736450195
epoch:0 | acc:1.0 | loss:0.007462739944458008
epoch:0 | acc:1.0 | loss:0.014298558235168457
epoch:0 | acc:1.0 | loss:0.017199933528900146
epoch:0 | acc:0.75 | loss:1.3312264680862427
epoch:0 | acc:1.0 | loss:0.008185088634490967
epoch:0 | acc:1.0 | loss:0.016142457723617554
epoch:0 | acc:1.0 | loss:0.01286858320236206
epoch:0 | acc:0.75 | loss:0.43040338158607483
epoch:0 | acc:1.0 | loss:0.02402395009994507
epoch:0 | acc:1.0 | loss:0.029601484537124634
epoch:0 | acc:1.0 | loss:0.03676971793

epoch:0 | acc:1.0 | loss:0.034144818782806396
epoch:0 | acc:1.0 | loss:0.015330910682678223
epoch:0 | acc:1.0 | loss:0.0216829776763916
epoch:0 | acc:1.0 | loss:0.019836366176605225
epoch:0 | acc:1.0 | loss:0.018334388732910156
epoch:0 | acc:1.0 | loss:0.027567028999328613
epoch:0 | acc:1.0 | loss:0.012422800064086914
epoch:0 | acc:0.75 | loss:0.8739170432090759
epoch:0 | acc:1.0 | loss:0.014597654342651367
epoch:0 | acc:1.0 | loss:0.02012425661087036
epoch:0 | acc:1.0 | loss:0.01926809549331665
epoch:0 | acc:1.0 | loss:0.020918548107147217
epoch:0 | acc:1.0 | loss:0.01791912317276001
epoch:0 | acc:1.0 | loss:0.015513777732849121
epoch:0 | acc:1.0 | loss:0.02306187152862549
epoch:0 | acc:1.0 | loss:0.019641995429992676
epoch:0 | acc:1.0 | loss:0.01423555612564087
epoch:0 | acc:1.0 | loss:0.013192474842071533
epoch:0 | acc:1.0 | loss:0.011683881282806396
epoch:0 | acc:1.0 | loss:0.020115554332733154
epoch:0 | acc:0.75 | loss:0.8040379285812378
epoch:0 | acc:1.0 | loss:0.0124017596244812

epoch:0 | acc:1.0 | loss:0.023946940898895264
epoch:0 | acc:1.0 | loss:0.01330837607383728
epoch:0 | acc:1.0 | loss:0.006888985633850098
epoch:0 | acc:1.0 | loss:0.010212153196334839
epoch:0 | acc:1.0 | loss:0.00796884298324585
epoch:0 | acc:1.0 | loss:0.009006470441818237
epoch:0 | acc:1.0 | loss:0.004964888095855713
epoch:0 | acc:1.0 | loss:0.03279213607311249
epoch:0 | acc:1.0 | loss:0.008937180042266846
epoch:0 | acc:1.0 | loss:0.0067389607429504395
epoch:0 | acc:1.0 | loss:0.005784451961517334
epoch:0 | acc:1.0 | loss:0.03187626600265503
epoch:0 | acc:1.0 | loss:0.005149126052856445
epoch:0 | acc:1.0 | loss:0.00509798526763916
epoch:0 | acc:1.0 | loss:0.006238400936126709
epoch:0 | acc:1.0 | loss:0.005605101585388184
epoch:0 | acc:0.75 | loss:0.19487716257572174
epoch:0 | acc:1.0 | loss:0.003276646137237549
epoch:0 | acc:1.0 | loss:0.015805423259735107
epoch:0 | acc:1.0 | loss:0.0054999589920043945
epoch:0 | acc:1.0 | loss:0.022080451250076294
epoch:0 | acc:1.0 | loss:0.0087240636

epoch:0 | acc:1.0 | loss:0.00632089376449585
epoch:0 | acc:1.0 | loss:0.01410934329032898
epoch:0 | acc:1.0 | loss:0.009158402681350708
epoch:0 | acc:1.0 | loss:0.008288562297821045
epoch:0 | acc:1.0 | loss:0.017168641090393066
epoch:0 | acc:1.0 | loss:0.008682727813720703
epoch:0 | acc:1.0 | loss:0.021484896540641785
epoch:0 | acc:1.0 | loss:0.007960349321365356
epoch:0 | acc:1.0 | loss:0.007923543453216553
epoch:0 | acc:1.0 | loss:0.009266138076782227
epoch:0 | acc:1.0 | loss:0.06962758302688599
epoch:0 | acc:1.0 | loss:0.006891727447509766
epoch:0 | acc:1.0 | loss:0.007310092449188232
epoch:0 | acc:1.0 | loss:0.007467120885848999
epoch:0 | acc:1.0 | loss:0.006304025650024414
epoch:0 | acc:1.0 | loss:0.008829474449157715
epoch:0 | acc:1.0 | loss:0.006835460662841797
epoch:0 | acc:1.0 | loss:0.009741723537445068
epoch:0 | acc:0.75 | loss:0.25152596831321716
epoch:0 | acc:1.0 | loss:0.005833923816680908
epoch:0 | acc:1.0 | loss:0.008397042751312256
epoch:0 | acc:1.0 | loss:0.0055575370

...

epoch:0 | acc:1.0 | loss:0.004068255424499512
epoch:0 | acc:1.0 | loss:0.006596207618713379
epoch:0 | acc:1.0 | loss:0.01110878586769104
epoch:0 | acc:1.0 | loss:0.018574774265289307
epoch:0 | acc:1.0 | loss:0.0047293901443481445
epoch:0 | acc:1.0 | loss:0.004323780536651611
epoch:0 | acc:1.0 | loss:0.006562232971191406
epoch:0 | acc:1.0 | loss:0.0037909746170043945
epoch:0 | acc:1.0 | loss:0.004332900047302246
epoch:0 | acc:1.0 | loss:0.00486069917678833
epoch:0 | acc:1.0 | loss:0.007985800504684448
epoch:0 | acc:1.0 | loss:0.004725217819213867
epoch:0 | acc:1.0 | loss:0.005504786968231201
epoch:0 | acc:1.0 | loss:0.00852280855178833
epoch:0 | acc:1.0 | loss:0.017870157957077026
epoch:0 | acc:1.0 | loss:0.009680509567260742
epoch:0 | acc:1.0 | loss:0.016098707914352417
epoch:0 | acc:1.0 | loss:0.030789881944656372
epoch:0 | acc:1.0 | loss:0.004774630069732666
epoch:0 | acc:1.0 | loss:0.008902162313461304
epoch:0 | acc:1.0 | loss:0.007097065448760986
epoch:0 | acc:1.0 | loss:0.01886969

epoch:0 | acc:1.0 | loss:0.005197346210479736
epoch:0 | acc:1.0 | loss:0.16800856590270996
epoch:0 | acc:1.0 | loss:0.04232838749885559
epoch:0 | acc:1.0 | loss:0.0052831172943115234
epoch:0 | acc:1.0 | loss:0.008033335208892822
epoch:0 | acc:1.0 | loss:0.0037105679512023926
epoch:0 | acc:1.0 | loss:0.0055852532386779785
epoch:0 | acc:1.0 | loss:0.008139342069625854
epoch:0 | acc:1.0 | loss:0.008014827966690063
epoch:0 | acc:1.0 | loss:0.00840449333190918
epoch:0 | acc:1.0 | loss:0.0076351165771484375
epoch:0 | acc:1.0 | loss:0.009659230709075928
epoch:0 | acc:1.0 | loss:0.005749762058258057
epoch:0 | acc:1.0 | loss:0.004809379577636719
epoch:0 | acc:1.0 | loss:0.006838202476501465
epoch:0 | acc:1.0 | loss:0.006309151649475098
epoch:0 | acc:1.0 | loss:0.014390379190444946
epoch:0 | acc:1.0 | loss:0.011154770851135254
epoch:0 | acc:1.0 | loss:0.0077263712882995605
epoch:0 | acc:1.0 | loss:0.008360475301742554
epoch:0 | acc:1.0 | loss:0.024320513010025024
epoch:0 | acc:1.0 | loss:0.00690

epoch:0 | acc:1.0 | loss:0.007468074560165405
epoch:0 | acc:1.0 | loss:0.004643738269805908
epoch:0 | acc:1.0 | loss:0.028866559267044067
epoch:0 | acc:0.75 | loss:0.7025078535079956
epoch:0 | acc:1.0 | loss:0.006461977958679199
epoch:0 | acc:1.0 | loss:0.004318535327911377
epoch:0 | acc:1.0 | loss:0.015159934759140015
epoch:0 | acc:1.0 | loss:0.013144195079803467
epoch:0 | acc:1.0 | loss:0.007957756519317627
epoch:0 | acc:1.0 | loss:0.008912861347198486
epoch:0 | acc:1.0 | loss:0.00440371036529541
epoch:0 | acc:1.0 | loss:0.003987491130828857
epoch:0 | acc:1.0 | loss:0.00639110803604126
epoch:0 | acc:1.0 | loss:0.004339039325714111
epoch:0 | acc:1.0 | loss:0.019652187824249268
epoch:0 | acc:1.0 | loss:0.006561458110809326
epoch:0 | acc:1.0 | loss:0.007726907730102539
epoch:0 | acc:1.0 | loss:0.012129902839660645
epoch:0 | acc:1.0 | loss:0.004225492477416992
epoch:0 | acc:1.0 | loss:0.007242560386657715
epoch:0 | acc:1.0 | loss:0.00395125150680542
epoch:0 | acc:1.0 | loss:0.01000970602

epoch:0 | acc:1.0 | loss:0.004758596420288086
epoch:0 | acc:1.0 | loss:0.0049304962158203125
epoch:0 | acc:1.0 | loss:0.03215564787387848
epoch:0 | acc:1.0 | loss:0.09762651473283768
epoch:0 | acc:1.0 | loss:0.005205750465393066
epoch:0 | acc:1.0 | loss:0.0077065229415893555
epoch:0 | acc:1.0 | loss:0.01020166277885437
epoch:0 | acc:1.0 | loss:0.10824349522590637
epoch:0 | acc:1.0 | loss:0.006628930568695068
epoch:0 | acc:1.0 | loss:0.007532298564910889
epoch:0 | acc:1.0 | loss:0.006157219409942627
epoch:0 | acc:1.0 | loss:0.00904381275177002
epoch:0 | acc:1.0 | loss:0.007460176944732666
epoch:0 | acc:1.0 | loss:0.013012230396270752
epoch:0 | acc:1.0 | loss:0.005010545253753662
epoch:0 | acc:1.0 | loss:0.009113818407058716
epoch:0 | acc:1.0 | loss:0.0052277445793151855
epoch:0 | acc:0.75 | loss:1.0032687187194824
epoch:0 | acc:0.75 | loss:1.0969860553741455
epoch:0 | acc:1.0 | loss:0.011956125497817993
epoch:0 | acc:1.0 | loss:0.0050778985023498535
epoch:0 | acc:0.75 | loss:0.961865544

epoch:0 | acc:1.0 | loss:0.014168530702590942
epoch:0 | acc:0.75 | loss:0.8547548651695251
epoch:0 | acc:1.0 | loss:0.009477883577346802
epoch:0 | acc:1.0 | loss:0.0042150020599365234
epoch:0 | acc:1.0 | loss:0.0049623847007751465
epoch:0 | acc:1.0 | loss:0.006120502948760986
epoch:0 | acc:0.75 | loss:0.9840179085731506
epoch:0 | acc:1.0 | loss:0.010898739099502563
epoch:0 | acc:1.0 | loss:0.005217790603637695
epoch:0 | acc:1.0 | loss:0.0032100677490234375
epoch:0 | acc:1.0 | loss:0.018144726753234863
epoch:0 | acc:1.0 | loss:0.010362565517425537
epoch:0 | acc:1.0 | loss:0.00663033127784729
epoch:0 | acc:1.0 | loss:0.01077917218208313
epoch:0 | acc:0.75 | loss:1.1223700046539307
epoch:0 | acc:1.0 | loss:0.00610804557800293
epoch:0 | acc:1.0 | loss:0.006352782249450684
epoch:0 | acc:1.0 | loss:0.007063448429107666
epoch:0 | acc:1.0 | loss:0.005942583084106445
epoch:0 | acc:1.0 | loss:0.006690263748168945
epoch:0 | acc:1.0 | loss:0.005297064781188965
epoch:0 | acc:1.0 | loss:0.0079087615

# 載入訓練好的模型並測試結果

In [20]:
model.load_state_dict(torch.load('bert_cla.ckpt'))

print('开始测试...')
model.eval()
test_result = []
for item in test_dataset:

    text_list = list(json.loads(item[0]))
    text_tensor = torch.tensor(text_list).unsqueeze(0).cuda()

    with torch.no_grad():

        # print('list', text_list)
        # print('tensor', text_tensor)
        # print('tensor.shape', text_tensor.shape)
        outputs = model(text_tensor, labels=None)

        print(outputs[0])

        pre = outputs[0].argmax(dim=1)
        test_result.append([item[0], pre.item()])

开始测试...
tensor([[ 3.0615, -2.7079]], device='cuda:0')
tensor([[-1.9948,  1.4176]], device='cuda:0')
tensor([[ 2.7601, -2.4883]], device='cuda:0')
tensor([[ 1.7807, -1.2395]], device='cuda:0')
tensor([[ 2.6653, -2.0854]], device='cuda:0')
tensor([[-2.2404,  1.9680]], device='cuda:0')
tensor([[-2.2323,  1.9231]], device='cuda:0')
tensor([[ 2.9455, -2.7364]], device='cuda:0')
tensor([[ 2.8297, -2.5776]], device='cuda:0')
tensor([[ 2.4902, -1.8415]], device='cuda:0')
tensor([[ 3.0042, -2.7335]], device='cuda:0')
tensor([[ 2.9092, -2.6257]], device='cuda:0')
tensor([[ 2.9912, -2.7522]], device='cuda:0')
tensor([[ 2.9797, -2.5760]], device='cuda:0')
tensor([[ 2.8414, -2.5982]], device='cuda:0')
tensor([[ 2.9830, -2.6804]], device='cuda:0')
tensor([[ 2.9783, -2.5576]], device='cuda:0')
tensor([[ 2.9783, -2.5576]], device='cuda:0')
tensor([[ 2.8223, -2.5544]], device='cuda:0')
tensor([[ 2.6408, -2.5349]], device='cuda:0')
tensor([[ 2.9961, -2.6345]], device='cuda:0')
tensor([[ 2.8784, -2.2536]

tensor([[ 2.9718, -2.6257]], device='cuda:0')
tensor([[-2.2441,  1.9230]], device='cuda:0')
tensor([[ 2.9692, -2.6709]], device='cuda:0')
tensor([[ 2.7475, -2.4376]], device='cuda:0')
tensor([[ 2.7927, -2.3282]], device='cuda:0')
tensor([[ 0.4549, -0.4818]], device='cuda:0')
tensor([[ 3.0394, -2.5635]], device='cuda:0')
tensor([[ 2.9783, -2.5576]], device='cuda:0')
tensor([[ 2.9330, -2.6665]], device='cuda:0')
tensor([[ 3.0476, -2.7428]], device='cuda:0')
tensor([[ 3.0779, -2.7099]], device='cuda:0')
tensor([[ 2.9779, -2.7036]], device='cuda:0')
tensor([[ 2.9783, -2.5576]], device='cuda:0')
tensor([[ 2.9783, -2.5576]], device='cuda:0')
tensor([[ 2.9783, -2.5576]], device='cuda:0')
tensor([[ 2.9034, -2.4493]], device='cuda:0')
tensor([[ 2.9641, -2.6142]], device='cuda:0')
tensor([[ 2.8225, -2.4341]], device='cuda:0')
tensor([[ 3.0741, -2.7194]], device='cuda:0')
tensor([[ 2.7601, -2.4883]], device='cuda:0')
tensor([[-2.2905,  1.9915]], device='cuda:0')
tensor([[ 2.9967, -2.5391]], devic

tensor([[-2.1584,  1.9090]], device='cuda:0')
tensor([[-2.2356,  1.9793]], device='cuda:0')
tensor([[ 2.7344, -2.2938]], device='cuda:0')
tensor([[ 2.9961, -2.6345]], device='cuda:0')
tensor([[-0.0753, -0.1548]], device='cuda:0')
tensor([[ 2.9783, -2.5576]], device='cuda:0')
tensor([[ 2.9348, -2.7138]], device='cuda:0')
tensor([[ 2.7436, -2.5451]], device='cuda:0')
tensor([[ 2.9339, -2.6246]], device='cuda:0')
tensor([[ 2.9935, -2.7252]], device='cuda:0')
tensor([[ 2.7212, -2.4576]], device='cuda:0')
tensor([[ 2.9491, -2.7330]], device='cuda:0')
tensor([[ 3.0209, -2.7351]], device='cuda:0')
tensor([[ 3.0447, -2.6782]], device='cuda:0')
tensor([[-1.8865,  1.4318]], device='cuda:0')
tensor([[ 2.9308, -2.7652]], device='cuda:0')
tensor([[ 2.5979, -2.3179]], device='cuda:0')
tensor([[ 1.1333, -0.7633]], device='cuda:0')
tensor([[ 3.0241, -2.7331]], device='cuda:0')
tensor([[ 2.9961, -2.6345]], device='cuda:0')
tensor([[ 2.9144, -2.6407]], device='cuda:0')
tensor([[ 2.9145, -2.4813]], devic

tensor([[ 2.8116, -2.4789]], device='cuda:0')
tensor([[-2.1996,  1.6818]], device='cuda:0')
tensor([[ 2.9896, -2.6582]], device='cuda:0')
tensor([[ 2.6401, -2.4508]], device='cuda:0')
tensor([[ 1.9730, -1.1346]], device='cuda:0')
tensor([[ 2.9731, -2.5871]], device='cuda:0')
tensor([[ 2.8840, -2.6359]], device='cuda:0')
tensor([[ 1.4706, -1.2306]], device='cuda:0')
tensor([[ 2.6229, -2.3125]], device='cuda:0')
tensor([[ 2.9967, -2.6345]], device='cuda:0')
tensor([[ 2.9026, -2.5535]], device='cuda:0')
tensor([[ 3.0440, -2.7570]], device='cuda:0')
tensor([[-2.3225,  1.9774]], device='cuda:0')
tensor([[ 2.8403, -2.5871]], device='cuda:0')
tensor([[ 3.0665, -2.7364]], device='cuda:0')
tensor([[ 2.6473, -2.0137]], device='cuda:0')
tensor([[ 2.9728, -2.6542]], device='cuda:0')
tensor([[ 2.9858, -2.7457]], device='cuda:0')
tensor([[ 2.9783, -2.5576]], device='cuda:0')
tensor([[ 3.0461, -2.6425]], device='cuda:0')
tensor([[ 2.7643, -2.6408]], device='cuda:0')
tensor([[-2.2042,  1.9679]], devic

tensor([[ 2.9440, -2.6454]], device='cuda:0')
tensor([[ 2.9783, -2.5576]], device='cuda:0')
tensor([[ 2.8030, -2.4404]], device='cuda:0')
tensor([[ 2.8795, -2.6087]], device='cuda:0')
tensor([[ 2.9981, -2.6866]], device='cuda:0')
tensor([[ 2.9783, -2.5576]], device='cuda:0')
tensor([[-1.9239,  1.3650]], device='cuda:0')
tensor([[ 3.0262, -2.7520]], device='cuda:0')
tensor([[ 2.6842, -2.4571]], device='cuda:0')
tensor([[ 2.9360, -2.5559]], device='cuda:0')
tensor([[ 2.8715, -2.1979]], device='cuda:0')
tensor([[ 2.7436, -2.5451]], device='cuda:0')
tensor([[ 2.8095, -2.4349]], device='cuda:0')
tensor([[ 3.0429, -2.7147]], device='cuda:0')
tensor([[ 2.9961, -2.6345]], device='cuda:0')
tensor([[-2.1978,  1.8748]], device='cuda:0')
tensor([[-0.4610,  0.2364]], device='cuda:0')
tensor([[-2.2861,  1.8415]], device='cuda:0')
tensor([[ 3.0499, -2.7255]], device='cuda:0')
tensor([[ 3.0647, -2.6768]], device='cuda:0')
tensor([[ 2.9783, -2.5576]], device='cuda:0')
tensor([[ 2.7708, -2.4477]], devic

...

tensor([[ 3.0073, -2.6844]], device='cuda:0')
tensor([[ 2.5264, -2.3478]], device='cuda:0')
tensor([[ 2.9728, -2.6542]], device='cuda:0')
tensor([[-2.2745,  2.0171]], device='cuda:0')
tensor([[ 2.9935, -2.7252]], device='cuda:0')
tensor([[ 2.8913, -2.5839]], device='cuda:0')
tensor([[ 2.5291, -2.0895]], device='cuda:0')
tensor([[ 2.3214, -1.6276]], device='cuda:0')
tensor([[-2.1812,  1.9574]], device='cuda:0')
tensor([[ 2.9692, -2.6709]], device='cuda:0')
tensor([[ 2.6397, -2.3108]], device='cuda:0')
tensor([[ 2.8228, -2.3619]], device='cuda:0')
tensor([[ 2.8093, -2.6267]], device='cuda:0')
tensor([[-2.0747,  1.5854]], device='cuda:0')
tensor([[-2.2604,  2.0132]], device='cuda:0')
tensor([[-2.1823,  1.8673]], device='cuda:0')
tensor([[ 2.9193, -2.5933]], device='cuda:0')
tensor([[-2.1938,  1.9546]], device='cuda:0')
tensor([[ 3.0383, -2.7111]], device='cuda:0')
tensor([[ 2.7104, -2.1184]], device='cuda:0')
tensor([[ 2.9465, -2.7243]], device='cuda:0')
tensor([[ 2.8299, -2.6636]], devic

tensor([[ 2.9692, -2.6709]], device='cuda:0')
tensor([[-2.2445,  1.7487]], device='cuda:0')
tensor([[ 0.7796, -0.5100]], device='cuda:0')
tensor([[ 2.9647, -2.7021]], device='cuda:0')
tensor([[ 2.3339, -1.7667]], device='cuda:0')
tensor([[ 2.9967, -2.6345]], device='cuda:0')
tensor([[ 0.9181, -0.7169]], device='cuda:0')
tensor([[ 2.8916, -2.6879]], device='cuda:0')
tensor([[ 2.9455, -2.5207]], device='cuda:0')
tensor([[ 2.8051, -2.6335]], device='cuda:0')
tensor([[ 2.7552, -2.3914]], device='cuda:0')
tensor([[ 3.0489, -2.7225]], device='cuda:0')
tensor([[ 3.0203, -2.7173]], device='cuda:0')
tensor([[ 3.0408, -2.7238]], device='cuda:0')
tensor([[ 2.9961, -2.6345]], device='cuda:0')
tensor([[ 3.0170, -2.6679]], device='cuda:0')
tensor([[-2.0955,  1.8051]], device='cuda:0')
tensor([[ 3.0984, -2.7065]], device='cuda:0')
tensor([[ 2.9961, -2.6345]], device='cuda:0')
tensor([[ 3.0131, -2.6481]], device='cuda:0')
tensor([[ 2.9311, -2.4531]], device='cuda:0')
tensor([[ 2.6651, -2.1272]], devic

tensor([[ 2.8429, -2.4846]], device='cuda:0')
tensor([[-2.1188,  1.4626]], device='cuda:0')
tensor([[ 2.8429, -2.4846]], device='cuda:0')
tensor([[-2.2096,  1.9328]], device='cuda:0')
tensor([[-2.0743,  1.7070]], device='cuda:0')
tensor([[-2.2074,  1.9561]], device='cuda:0')
tensor([[-1.9483,  1.4093]], device='cuda:0')
tensor([[ 2.7632, -2.4524]], device='cuda:0')
tensor([[ 2.9783, -2.5576]], device='cuda:0')
tensor([[ 2.9716, -2.6370]], device='cuda:0')
tensor([[ 3.0188, -2.6826]], device='cuda:0')
tensor([[ 2.9372, -2.7112]], device='cuda:0')
tensor([[ 2.7293, -1.9851]], device='cuda:0')
tensor([[ 3.0453, -2.7308]], device='cuda:0')
tensor([[ 3.0758, -2.6660]], device='cuda:0')
tensor([[ 2.6104, -2.0568]], device='cuda:0')
tensor([[ 3.0215, -2.7093]], device='cuda:0')
tensor([[-2.3146,  1.8559]], device='cuda:0')
tensor([[ 2.8031, -2.6132]], device='cuda:0')
tensor([[ 2.9288, -2.6605]], device='cuda:0')
tensor([[ 2.9830, -2.6804]], device='cuda:0')
tensor([[ 2.9783, -2.5576]], devic

tensor([[ 3.0476, -2.7428]], device='cuda:0')
tensor([[ 2.9026, -2.5535]], device='cuda:0')
tensor([[ 3.0631, -2.6767]], device='cuda:0')
tensor([[ 2.9134, -2.5470]], device='cuda:0')
tensor([[ 2.8131, -2.4803]], device='cuda:0')
tensor([[ 2.6735, -2.4583]], device='cuda:0')
tensor([[ 2.9692, -2.6709]], device='cuda:0')
tensor([[ 3.0209, -2.7351]], device='cuda:0')
tensor([[-2.1861,  1.9353]], device='cuda:0')
tensor([[ 2.9180, -2.6867]], device='cuda:0')
tensor([[ 2.9783, -2.5576]], device='cuda:0')
tensor([[ 2.7475, -2.5572]], device='cuda:0')
tensor([[ 3.0205, -2.7314]], device='cuda:0')
tensor([[ 2.7890, -2.5044]], device='cuda:0')
tensor([[ 2.8595, -2.5818]], device='cuda:0')
tensor([[ 2.6201, -2.2808]], device='cuda:0')
tensor([[ 2.9416, -2.6674]], device='cuda:0')
tensor([[ 3.0142, -2.7013]], device='cuda:0')
tensor([[ 3.0201, -2.7035]], device='cuda:0')
tensor([[-2.2731,  1.8816]], device='cuda:0')
tensor([[-0.0862, -0.1439]], device='cuda:0')
tensor([[ 2.8651, -2.5854]], devic

tensor([[ 2.6118, -2.2171]], device='cuda:0')
tensor([[ 2.5625, -1.7838]], device='cuda:0')
tensor([[-2.2435,  1.8993]], device='cuda:0')
tensor([[ 2.9428, -2.6255]], device='cuda:0')
tensor([[ 2.9783, -2.5576]], device='cuda:0')
tensor([[ 2.9336, -2.3072]], device='cuda:0')
tensor([[ 2.7828, -2.4927]], device='cuda:0')
tensor([[-1.9160,  1.5027]], device='cuda:0')
tensor([[ 3.0422, -2.7685]], device='cuda:0')
tensor([[ 2.7344, -2.2938]], device='cuda:0')
tensor([[ 2.9731, -2.5871]], device='cuda:0')
tensor([[ 2.7344, -2.2938]], device='cuda:0')
tensor([[ 2.7087, -2.2475]], device='cuda:0')
tensor([[ 1.6538, -1.3599]], device='cuda:0')
tensor([[ 3.0309, -2.7142]], device='cuda:0')
tensor([[ 2.7632, -2.4524]], device='cuda:0')
tensor([[ 3.0667, -2.7331]], device='cuda:0')
tensor([[ 2.4727, -1.8085]], device='cuda:0')
tensor([[ 2.9471, -2.5698]], device='cuda:0')
tensor([[ 2.9183, -2.6251]], device='cuda:0')
tensor([[-1.7162,  1.3738]], device='cuda:0')
tensor([[ 2.7467, -2.1236]], devic

text_list

In [64]:
test_result

[['[101, 118, 8211, 0, 0, 0, 0, 0, 0, 0, 0, 102]', 0],
 ['[101, 2544, 5131, 120, 4229, 120, 4397, 4403, 1048, 6527, 0, 102]', 1],
 ['[101, 769, 3211, 3209, 5169, 0, 0, 0, 0, 0, 0, 102]', 0],
 ['[101, 1912, 2380, 0, 0, 0, 0, 0, 0, 0, 0, 102]', 0],
 ['[101, 1366, 0, 0, 0, 0, 0, 0, 0, 0, 0, 102]', 0],
 ['[101, 2875, 4277, 3862, 7804, 5114, 0, 0, 0, 0, 0, 102]', 1],
 ['[101, 5429, 5428, 2831, 7619, 0, 0, 0, 0, 0, 0, 102]', 1],
 ['[101, 8439, 120, 8111, 120, 8110, 0, 0, 0, 0, 0, 102]', 0],
 ['[101, 802, 4412, 131, 0, 0, 0, 0, 0, 0, 0, 102]', 0],
 ['[101, 1066, 123, 7517, 1912, 2380, 131, 0, 0, 0, 0, 102]', 0],
 ['[101, 8150, 118, 8211, 0, 0, 0, 0, 0, 0, 0, 102]', 0],
 ['[101, 109, 8290, 0, 0, 0, 0, 0, 0, 0, 0, 102]', 0],
 ['[101, 8113, 131, 8259, 0, 0, 0, 0, 0, 0, 0, 102]', 0],
 ['[101, 4634, 4873, 5998, 131, 100, 0, 0, 0, 0, 0, 102]', 0],
 ['[101, 7032, 7540, 0, 0, 0, 0, 0, 0, 0, 0, 102]', 0],
 ['[101, 116, 0, 0, 0, 0, 0, 0, 0, 0, 0, 102]', 0],
 ['[101, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 102]

In [26]:
test_data = pd.read_csv('/home/jacklee/ocr/food_classification/train_data/test_data.csv')

In [27]:
test_data

,text,label
0,-32,0
1,微糖/熱/珍珠免費,0
2,交易明細,0
3,外帶,0
4,口,0
...,...,...
5394,1460.0,0
5395,零:,0
5396,交易明細,0
5397,澳洲洋葱(大),1


In [65]:
test_data['pre'] = 0
for i in range(len(test_dataset)):
    test_data['pre'][i] = test_result[i][1]

/home/jacklee/.conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [70]:
a=0
for i in range(len(test_dataset)):   
    if test_data['label'][i] != test_data['pre'][i]:
        a+=1